In [1]:
import gist

from PIL import Image
import imageio
from IPython.display import display_png, clear_output

import sys
sys.path.append("../Library/")
import image_download as imd
import image_manipulation as ima

import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from dotenv import load_dotenv
import os
load_dotenv("../.env")
import csv

import numpy as np
import db_connection as dbcon

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime

In [3]:
def get_filenames_unlabeled_images(filenames, labeled_images):
    unlabeled = list(set(filenames) - set(labeled_images))
    return unlabeled

def label_images(folder, filenames, num_images):
    random.shuffle(filenames)
    
    labels = {}
    for i in range(num_images):
        imarray = np.asarray(Image.open(folder + filenames[i]))[:,:,:3]
        imarray_copy = imarray.copy()
        
        # Vertical and horizontal line at 1/3
        imarray_copy[int(imarray_copy.shape[0]/3),:,:3] = 0
        imarray_copy[:,int(imarray_copy.shape[0]/3),:3] = 0
        
        # Vertical and horizontal line at 2/3
        imarray_copy[int(2*imarray_copy.shape[0]/3),:,:3] = 0
        imarray_copy[:,int(2*imarray_copy.shape[0]/3),:3] = 0
        
        im = Image.fromarray(imarray_copy)
        display_png(im)
        labels[filenames[i]] = input("Label:")
        clear_output()
        
    return labels

### Set Parameters

In [4]:
# parameters
MFP_FOLDER = os.getenv('MFP_FOLDER')
EX_FOLDER = MFP_FOLDER + 'earthexplorer/'
IMAGE_FOLDER = EX_FOLDER + 'processedIMG/'
LABELS_FOLDER = EX_FOLDER + 'labels/'

### List all image filenames

In [5]:
filenames =  os.listdir(IMAGE_FOLDER)
filenames[-10:]

['m_4209830_se_14_1_20140907_x4096_y1536.png',
 'm_4209830_se_14_1_20140907_x1536_y0.png',
 'm_3409456_ne_15_1_20170914_x4096_y5120.png',
 'm_4209725_sw_14_1_20140917_x5632_y4096.png',
 'm_2608164_sw_17_1_20171210_x6144_y6144.png',
 'm_2608164_sw_17_1_20171210_x6656_y3584.png',
 'm_3809356_nw_15_1_20140703_x6656_y1536.png',
 'm_3409456_ne_15_1_20170914_x6656_y3584.png',
 'm_3809356_nw_15_1_20140703_x6144_y512.png',
 'm_4209830_se_14_1_20140907_x1024_y4608.png']

### List images that are already labeled

### Label images

In [58]:
labels_csv = os.listdir(LABELS_FOLDER)
labels_csv = [label_csv for label_csv in labels_csv if label_csv.startswith("labels")]

labeled_img = []
for fname_csv in labels_csv:
    df = pd.read_csv(LABELS_FOLDER + fname_csv)
    for fname in df.filename:
        labeled_img.append(fname)

unlabeled = get_filenames_unlabeled_images(filenames, labeled_img)
print("Number of unlabeled images:", len(unlabeled))
print("Number of labeled images:", len(labeled_img))

Number of unlabeled images: 1859
Number of labeled images: 80


In [56]:
labels = label_images(IMAGE_FOLDER, unlabeled, 20)
df_labels = pd.DataFrame({
        'filename': list(labels.keys()),
        'label': list(labels.values())
})

In [57]:
df_labels.to_csv(LABELS_FOLDER + "/labels_" + str(datetime.now()) + ".csv")